In [14]:
import random
import cv2, os
import pandas as pd
import numpy as np
import math

from matplotlib import pyplot as plt

base_path = "D:/Developer/2021___1/AI/Datasets/pomeranian"
origin_path = "pome_train"
rot_path = "rot_pome_train"

file_list = sorted(os.listdir(base_path + "/" + origin_path))
print(file_list)
print('size = ', len(file_list))

try:
  os.mkdir(base_path+"/"+rot_path)
except:
  print("directory exist")

jpg', 'pome_513.jpg.dog', 'pome_513.xml', 'pome_514.jpg', 'pome_514.jpg.dog', 'pome_514.xml', 'pome_515.jpg', 'pome_515.jpg.dog', 'pome_515.xml', 'pome_516.jpg', 'pome_516.jpg.dog', 'pome_516.xml', 'pome_517.jpg', 'pome_517.jpg.dog', 'pome_517.xml', 'pome_518.jpg', 'pome_518.jpg.dog', 'pome_518.xml', 'pome_519.jpg', 'pome_519.jpg.dog', 'pome_519.xml', 'pome_520.jpg', 'pome_520.jpg.dog', 'pome_520.xml', 'pome_521.jpg', 'pome_521.jpg.dog', 'pome_521.xml', 'pome_522.jpg', 'pome_522.jpg.dog', 'pome_522.xml', 'pome_523.jpg', 'pome_523.jpg.dog', 'pome_523.xml', 'pome_524.jpg', 'pome_524.jpg.dog', 'pome_524.xml', 'pome_525.jpg', 'pome_525.jpg.dog', 'pome_525.xml', 'pome_526.jpg', 'pome_526.jpg.dog', 'pome_526.xml', 'pome_527.jpg', 'pome_527.jpg.dog', 'pome_527.xml', 'pome_528.jpg', 'pome_528.jpg.dog', 'pome_528.xml', 'pome_529.jpg', 'pome_529.jpg.dog', 'pome_529.xml', 'pome_530.jpg', 'pome_530.jpg.dog', 'pome_530.xml', 'pome_531.jpg', 'pome_531.jpg.dog', 'pome_531.xml', 'pome_532.jpg', 'pome_

In [8]:
import xml.etree.ElementTree as Et
from xml.etree.ElementTree import Element, ElementTree

def Rotate_Bound(image, angle):
  (h, w) = image.shape[:2]
  (cX, cY) = (w // 2, h // 2)
  M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
  cos = np.abs(M[0, 0])
  sin = np.abs(M[0, 1])

  nW = int((h * sin) + (w * cos))
  nH = int((h * cos) + (w * sin))

  M[0, 2] += (nW / 2) - cX
  M[1, 2] += (nH / 2) - cY

  return cv2.warpAffine(image, M, (nW, nH)), M[0, -1], M[-1, -1]

def Rotate_Point(angle, x, y):
    a = np.radians(angle)
    cosa = np.cos(a)
    sina = np.sin(a)

    return x*cosa - y*sina, x*sina + y*cosa

def getBoundingBoxData(origin_filename):
  try:
    tree = Et.parse(os.path.join(base_path + "/" + origin_path, origin_filename + ".xml"))
  except:
    return 0,0,0,0

  root = tree.getroot()

  objects = root.findall("object")
  bb = []

  if len(objects) > 1:
    return 0,0,0,0

  for _object in objects:
    name = _object.find("name").text
    bndbox = _object.find("bndbox")
    xmin = (int)(bndbox.find("xmin").text)
    ymin = (int)(bndbox.find("ymin").text)
    xmax = (int)(bndbox.find("xmax").text)
    ymax = (int)(bndbox.find("ymax").text)
    break

  return xmin, ymin, xmax, ymax

def getBoundingBoxDataCheck(origin_filename):
  ## Box 데이터 확인 체크시 사용한다
  print("getBoundingBoxDataCheck : ", os.path.join(base_path + "/" + rot_path, origin_filename + ".xml"))
  try:
    tree = Et.parse(os.path.join(base_path + "/" + rot_path, origin_filename + ".xml"))
  except:
    return 0,0,0,0

  root = tree.getroot()

  objects = root.findall("object")
  bb = []

  if len(objects) > 1:
    return 0,0,0,0

  for _object in objects:
    name = _object.find("name").text
    bndbox = _object.find("bndbox")
    xmin = (int)(bndbox.find("xmin").text)
    ymin = (int)(bndbox.find("ymin").text)
    xmax = (int)(bndbox.find("xmax").text)
    ymax = (int)(bndbox.find("ymax").text)
    break

  return xmin, ymin, xmax, ymax

def writeXML(origin_filename, str_anglename, img_filename, new_width, new_height, xmin, ymin, xmax, ymax):
  try:
    tree = Et.parse(os.path.join(base_path + "/" + origin_path, origin_filename + ".xml"))
  except:
    return

  root = tree.getroot()

  root.find("filename").text = str_anglename + img_filename
  sizes = root.findall("size")
  objects = root.findall("object")
  bb = []

  for _size in sizes:
    _size.find("width").text = str(new_width)
    _size.find("height").text = str(new_height)
    break

  if len(objects) > 1:
    return

  for _object in objects:
    # name = _object.find("name").text
    bndbox = _object.find("bndbox")

    bndbox.find("xmin").text = str((int)xmin)
    bndbox.find("ymin").text = str((int)ymin)
    bndbox.find("xmax").text = str((int)xmax)
    bndbox.find("ymax").text = str((int)ymax)

    break

  # print("xml save name : ", os.path.join(base_path, "rot_" + origin_filename + ".xml"))
  tree.write(os.path.join(base_path + "/" + rot_path, str_anglename + origin_filename + ".xml"), encoding='utf8' , xml_declaration=False)
  print(os.path.join(base_path + "/" + rot_path, str_anglename + origin_filename + ".xml"))

  return

def getRatioImagePointSize(img):
  height, width, channels = img.shape
  ratioSize = width if width > height else height

  return ratioSize

def adjustBoundingBox(angle, xmin, ymin, xmax, ymax):
  xmin += ratio_x
  xmax += ratio_x
  ymin += ratio_y
  ymax += ratio_y

  return xmin, ymin, xmax, ymax

In [15]:
num = 0
angles = [-30, -25, -20, -15, -10, -5, 5, 10, 15, 20, 25, 30]
# angles = [25, 30]

for angle in angles:
  # angle = 20
  str_anglename = 'P' if angle > 0 else 'M'
  str_anglename += str(abs(angle)) + "_rot_"
  print("str_anglename : ", str_anglename)
  print("Save directory : ", base_path+"/"+str_anglename+rot_path)

  for f in file_list:
    if '.dog' not in f:
      continue

    num = num + 1

    pd_frame = pd.read_csv(os.path.join(base_path + "/" + origin_path, f), sep=' ', header=None)
    np_landmarks = pd_frame.to_numpy()
    landmarks = (np_landmarks[0][1:-1]).reshape((-1, 2)).astype(np.int)
    img_filename, ext = os.path.splitext(f)
    origin_filename, origin_ext = os.path.splitext(img_filename)

    print(img_filename)
    if len(landmarks) != 3:
      continue

    img = cv2.imread(os.path.join(base_path  + "/" + origin_path, img_filename))
    rot_img, ratio_x, ratio_y = Rotate_Bound(img, angle) # 회전추가

    ## Bounding box 가져오기
    xmin, ymin, xmax, ymax = getBoundingBoxData(origin_filename)
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 3)

    if xmin == 0 and ymin == 0 and xmax == 0 and ymax == 0:
      continue

    rot_xmin, rot_ymin = Rotate_Point(-angle, xmin, ymin)
    rot_xmax, rot_ymax = Rotate_Point(-angle, xmax, ymax)

    reverse_xmin, reverse_ymin = Rotate_Point(-angle, xmax, ymin)
    reverse_xmax, reverse_ymax = Rotate_Point(-angle, xmin, ymax)
    
    rot_xmin, rot_ymin, rot_xmax, rot_ymax = adjustBoundingBox(angle, rot_xmin, rot_ymin, rot_xmax, rot_ymax)
    reverse_xmin, reverse_ymin, reverse_xmax, reverse_ymax = adjustBoundingBox(angle, reverse_xmin, reverse_ymin, reverse_xmax, reverse_ymax)

    ratioSize = getRatioImagePointSize(img)

    if angle > 0:
      new_xmin = round(rot_xmin)
      new_ymin = round(reverse_ymin)
      new_xmax = round(rot_xmax)
      new_ymax = round(reverse_ymax)
    else:
      new_xmin = round(reverse_xmax)
      new_ymin = round(rot_ymin)
      new_xmax = round(reverse_xmin)
      new_ymax = round(rot_ymax)
      
    
    str_new_landmark = str(len(landmarks))
    

    for idx, l in enumerate(landmarks):
      x2, y2 = Rotate_Point(-angle, l[0], l[1])
      x2 += ratio_x
      y2 += ratio_y

      str_new_landmark += ' ' + str(round(x2)) + ' ' + str(round(y2))
      
      cv2.circle(img, center=tuple(l), radius=round(ratioSize/80), color=(0,255,0), thickness=-1)
      cv2.putText(img, str(idx+1), tuple(l), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 0.75, (255, 255, 255), 2, cv2.LINE_AA)
      # cv2.circle(rot_img, (round(x2), round(y2)), radius=round(ratioSize/80), color=(0,0,255), thickness=-1)
      # cv2.putText(rot_img, str(idx+1), (round(x2), round(y2)), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    str_new_landmark += ' '

    if cv2.waitKey(0) == ord('q'):
      break

    save_path = base_path + "/" + rot_path + "/" + str_anglename + img_filename
    cv2.imwrite(save_path, rot_img, [int(cv2.IMWRITE_JPEG_QUALITY), 50]) # 저장 추가

    rot_height, rot_width, channels = rot_img.shape
    writeXML(origin_filename, str_anglename, img_filename, rot_width, rot_height, new_xmin, new_ymin, new_xmax, new_ymax)

    print(save_path)

    with open(save_path + '.dog', 'w') as writefile:
      writefile.write(str_new_landmark)



Datasets/pomeranian/rot_pome_train/P30_rot_pome_877.jpg
pome_878.jpg
D:/Developer/2021___1/AI/Datasets/pomeranian/rot_pome_train\P30_rot_pome_878.xml
D:/Developer/2021___1/AI/Datasets/pomeranian/rot_pome_train/P30_rot_pome_878.jpg
pome_879.jpg
D:/Developer/2021___1/AI/Datasets/pomeranian/rot_pome_train\P30_rot_pome_879.xml
D:/Developer/2021___1/AI/Datasets/pomeranian/rot_pome_train/P30_rot_pome_879.jpg
pome_880.jpg
D:/Developer/2021___1/AI/Datasets/pomeranian/rot_pome_train\P30_rot_pome_880.xml
D:/Developer/2021___1/AI/Datasets/pomeranian/rot_pome_train/P30_rot_pome_880.jpg
pome_881.jpg
D:/Developer/2021___1/AI/Datasets/pomeranian/rot_pome_train\P30_rot_pome_881.xml
D:/Developer/2021___1/AI/Datasets/pomeranian/rot_pome_train/P30_rot_pome_881.jpg
pome_882.jpg
D:/Developer/2021___1/AI/Datasets/pomeranian/rot_pome_train\P30_rot_pome_882.xml
D:/Developer/2021___1/AI/Datasets/pomeranian/rot_pome_train/P30_rot_pome_882.jpg
pome_884.jpg
D:/Developer/2021___1/AI/Datasets/pomeranian/rot_pome_tr